In [89]:
# !pip install google-search-results
import pandas as pd
# api_key = "e4fb6e74a565645df616278e366b117d5d63cc04d23182ce7fca8c25ec2fda1b"


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from serpapi import GoogleSearch
import json

In [14]:
search_term = "Data Science"
search_location = "India"
params = {
    "api_key" : api_key,
    "device" : "desktop",
    "engine" : "google_jobs",
    "q" : search_term,
    "hl": 'en',
    'gl': 'in',
    "start" : 0 #pagination
}

search = GoogleSearch(params)

In [125]:
data = search.get_dict()

https://serpapi.com/search


In [126]:
data.keys()

dict_keys(['search_metadata', 'search_parameters', 'jobs_results', 'chips'])

In [127]:
data['search_metadata']

{'id': '63f0c4d6c9de45d8b1c59380',
 'status': 'Success',
 'json_endpoint': 'https://serpapi.com/searches/bf48ecef481ef330/63f0c4d6c9de45d8b1c59380.json',
 'created_at': '2023-02-18 12:30:14 UTC',
 'processed_at': '2023-02-18 12:30:14 UTC',
 'google_jobs_url': 'https://www.google.com/search?q=Data+Science&ibp=htl;jobs&hl=en&start=40',
 'raw_html_file': 'https://serpapi.com/searches/bf48ecef481ef330/63f0c4d6c9de45d8b1c59380.html',
 'total_time_taken': 1.46}

In [20]:
with open(r"C:\Users\Jadha\PycharmProjects\python-stuff\sample.json", "w") as outfile:
    json.dump(data['jobs_results'], outfile)

In [128]:
len(data['jobs_results'])

10

In [129]:
data['jobs_results']

[{'title': 'Senior Data Science Engineer | Chicago, IL',
  'company_name': 'HopHR',
  'location': 'Chicago, IL',
  'via': 'via CareerBuilder',
  'description': "POSITION SUMMARY:\nWe're looking for a Senior Data Scientist in our growing Enterprise Data Science Team to develop new data science products using machine learning, natural language processing, data visualization techniques, and other forms of advanced analytics\n...\nThe Enterprise Data Science team is also a Center of Excellence for data science at our company. This person will spend approximately a quarter of their time promoting best practices in data science across the enterprise and leading and organizing technical training for the broader community.\n\nAt the largest customer-owned and not-for-profit health insurer and fourth largest health insurer overall in the United States, you will have the opportunity to work with massive datasets to drive revenue growth, improve our operational and member-facing processes, and af

In [24]:
job = data['jobs_results'][0]

In [26]:
print(job.keys())

dict_keys(['title', 'company_name', 'location', 'via', 'description', 'job_highlights', 'related_links', 'thumbnail', 'extensions', 'detected_extensions', 'job_id'])


In [52]:
print(job['title'])
print(job['company_name'])
print(job['location'])
print(job['via'])
# print(job['description']) # contails all the textual stuff
print([highlight['title'] for highlight in job['job_highlights']]) # Short of description
print(job['related_links'])
print(job['extensions'])
print(job['detected_extensions'])
# job['job_id']

Data Scientist II
Merck KGaA Darmstadt, Germany
St. Louis, MO
via EMD Group
['Qualifications', 'Responsibilities']
[{'link': 'https://www.emdgroup.com/', 'text': 'emdgroup.com'}, {'link': 'https://www.google.com/search?hl=en&q=Merck+KGaA+Darmstadt,+Germany&sa=X&ved=0ahUKEwi-i66v_579AhVqmGoFHVNqCvYQmJACCMwJ', 'text': 'See web results for Merck KGaA Darmstadt, Germany'}]
['1 day ago', 'Full-time']
{'posted_at': '1 day ago', 'schedule_type': 'Full-time'}


In [55]:
job = data['jobs_results'][4]

In [123]:
job

{'title': 'Director, Data Science',
 'company_name': 'Upside Services',
 'location': 'Sierra Vista, AZ',
 'via': 'via WANE Jobs',
 'description': "Meet Upside:\n\nWe created Upside to help communities thrive! Our retail technology uses the sophistication of online retail-profit measurement, attribution, and incrementality-to provide users with more value on their everyday purchases and brick and mortar businesses with new, profitable customers. We've helped millions of users earn 2 to 3 times more cash back than any other product, and tens of... thousands of brick and mortar businesses earn measurable profit. Billions of dollars in commerce run through the Upside platform every year, and that value goes directly back to our local retailers, the consumers they serve, and towards important sustainability initiatives. Our mission, values, and commitment to inclusivity guide our team of more than 300 people worldwide, and the quality of our culture is reflected in the impact we've had on c

In [56]:
print(job['title'])
print(job['company_name'])
print(job['location'])
print(job['via'])
# print(job['description']) # contails all the textual stuff
print([highlight['title'] for highlight in job['job_highlights']]) # Short of description
print(job['related_links'])
print(job['extensions'])
print(job['detected_extensions'])

Director of Data Science, Expansion - Slack
Salesforce
Denver, CO
via LinkedIn
['Qualifications', 'Responsibilities', 'Benefits']
[{'link': 'http://www.salesforce.com/', 'text': 'salesforce.com'}, {'link': 'https://www.google.com/search?hl=en&q=Salesforce&sa=X&ved=0ahUKEwi-i66v_579AhVqmGoFHVNqCvYQmJACCOIL', 'text': 'See web results for Salesforce'}]
['1 day ago', 'Full-time', 'Health insurance']
{'posted_at': '1 day ago', 'schedule_type': 'Full-time'}


## Get Combine Json data for 5 pages for dataScience

In [65]:
metadata = []
results = []

for i in range(5):
    search = GoogleSearch({**params, 'start' : i*10})
    page_data = search.get_dict()
    metadata.append(page_data['search_metadata'])
    results += page_data['jobs_results']

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


## Create CSV file
extra features can be added like `employee website`

`Key Points` - Some day ago | fulltime or PartTime | worktype

In [115]:
data = {
    'title': [],
    'company_name': [],
    'location': [],
    'from': [],
    'Benefits': [],
    'Qualifications': [],
    'Responsibilities': [],
}

highlight_keys = ['Benefits', 'Qualifications', 'Responsibilities']

for job in results:
    data['title'].append(job['title'])
    data['company_name'].append(job['company_name'])
    data['location'].append(job['location'])
    data['from'].append(job['via'])
    
    for k in highlight_keys:
        found = False
        for h in job['job_highlights']:
            if 'title' in h.keys() and k == h['title']:
                data[k].append(".".join(h['items']))
                found = True
        if not found:
            data[k].append("")


In [116]:
df = pd.DataFrame.from_dict(data)
df.to_csv('main.csv', index=False)

In [130]:
df

,title,company_name,location,from,Benefits,Qualifications,Responsibilities
0,Senior Data Scientist - NLP,Chewy,"Plantation, FL",via Chewy Careers,,"Bachelors’ degree with 5+ years’, Master's deg...",You will play a pivotal role in building new n...
1,Strategic Analytics Lead Data Scientist,Deloitte,"Fort Lauderdale, FL",via LinkedIn,,"Bachelors' / Masters' Degree in Statistics, Ap...",The Strategic Analytics team typically execute...
2,"Associate Director, Data Science",Chewy,"Dania Beach, FL",via WREG Jobs,,"MS in data science, economics, computer scienc...",Meet the Merchandising Data Science and Econom...
3,Vice President - Lead AML Data Science Analyst...,Citi,"Tampa, FL",via WJTV Jobs,"Primary Location Salary Range:$105,850.00 - $1...","The ideal candidate is a problem solver, equip...",As a Data Intelligence & Analytics Lead Data S...
4,Data Scientist (2211),SMX,"Tampa, FL",via SimplyHired,,Required clearance: Top Secret.Successful cand...,Develop customized data extraction and collati...
5,Data Scientist,Envision,Anywhere,via LinkedIn,,"MS Computer Science, Engineering, Technology, ...",
6,Data Science Analyst,American Heart Association,"West Palm Beach, FL",via CareerBuilder,Compensation & Benefits.Visit Rewards & Benefi...,Proficient to excellent in coding skills inclu...,This position provides opportunities to work w...
7,Manager Data Science CoE,Cardinal Health,"Dublin, OH (+1 other)",via Cardinal Health,Anticipated salary range: $100k-$160k.Bonus el...,3+ years in the Healthcare industry and knowle...,They will work closely with business stakehold...
8,Senior Data Scientist - Delivery Technology (R...,Gopuff,Anywhere,via Built In,Gopuff pays employees based on market pricing ...,3-5 years of experience in data science.MS or ...,Collaborate with our business and engineering ...
9,High Growth Tech Start Up - Senior Data Scient...,Hudson Modeling,Anywhere,via LinkedIn,,We seek a Senior Data Scientist (5+ yrs.You mu...,Brands with a microscopic understanding of the...
